# HAB Capstone Data Wrangling 3 - Ohio EPA Data

In [12]:
import folium
import matplotlib.pyplot as plt
import matplotlib.style
import numpy as np
import pandas as pd
import seaborn

In [13]:
oh_df = pd.read_csv('../data/interim/oh_epa.csv', index_col=0, parse_dates = ['Collect Date'])
oh_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 886 entries, 0 to 885
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Site             886 non-null    object        
 1   Site ID          886 non-null    object        
 2   Site Type        886 non-null    object        
 3   Collect Date     886 non-null    datetime64[ns]
 4   Collection Time  886 non-null    object        
 5   Sample ID        886 non-null    object        
 6   Parameter        886 non-null    object        
 7   Result           886 non-null    float64       
 8   Units            886 non-null    object        
 9   Qual.*           193 non-null    object        
 10  Program          537 non-null    object        
 11  Lab              886 non-null    object        
 12  Lat              886 non-null    float64       
 13  Long             886 non-null    float64       
 14  Saturation       886 non-null    bool     

In [14]:
oh_df.head()

,Site,Site ID,Site Type,Collect Date,Collection Time,Sample ID,Parameter,Result,Units,Qual.*,Program,Lab,Lat,Long,Saturation
0,LAKE ERIE UTILITIES CO 2-TP-LT2001-Raw,6262096-LT2001,PWS Plant Intake Site,2018-12-25,00:00,2121629,Microcystins,0.0,ug/L,NaN,NaN,OSU STONE LABORATORY,41.689305,-82.817555,False
1,LAKE ERIE UTILITIES CO 2-TP-LT2001-Raw,6262096-LT2001,PWS Plant Intake Site,2018-12-10,00:00,2115433,Microcystins,0.0,ug/L,NaN,NaN,OSU STONE LABORATORY,41.689305,-82.817555,False
2,LAKE ERIE UTILITIES CO 2-TP-LT2001-Raw,6262096-LT2001,PWS Plant Intake Site,2018-11-26,00:00,2107254,Microcystins,0.0,ug/L,NaN,NaN,OSU STONE LABORATORY,41.689305,-82.817555,False
3,LAKE ERIE UTILITIES CO 2-TP-LT2001-Raw,6262096-LT2001,PWS Plant Intake Site,2018-11-12,00:00,2103289,Microcystins,0.0,ug/L,NaN,NaN,OSU STONE LABORATORY,41.689305,-82.817555,False
4,Lake Erie Utilities WTP,6262096-EP002,PWS Finished Water Site,2018-10-29,00:00,2094113,Microcystins,0.0,ug/L,NaN,NaN,OSU STONE LABORATORY,41.689305,-82.817555,False


In [15]:
oh_df['Site Type'].value_counts()

Lake Site                  536
PWS Plant Intake Site      196
PWS Finished Water Site    154
Name: Site Type, dtype: int64

In [17]:
oh_df['Qual.*'].value_counts()

U-       123
U+        40
J         12
U-,UJ      9
U-,J       4
PS         3
U+,J       1
PS,J       1
Name: Qual.*, dtype: int64

In [18]:
oh_df['Parameter'].value_counts()

Microcystins          869
Saxitoxin               9
Cylindrospermopsin      8
Name: Parameter, dtype: int64

In [19]:
#select only measurements related to microcystin
oh_df = oh_df[oh_df['Parameter'] == 'Microcystins'].copy()

In [20]:
oh_df['Site'].value_counts()

LAKE ERIE UTILITIES CO 2-TP-LT2001-Raw            195
Lake Erie Utilities WTP                           154
Maumee Bay State Park Lake Erie Beach              64
Lake Erie Ambient Site - Off Maumee Bay            59
Lake Erie North of Port Clinton                    34
Lake Erie Ambient Station - West Sister Island     34
Lake Erie Ambient Site - Port Clinton              33
Lake Erie between Toledo/Oregon WTP Intakes        32
Lake Erie Ambient Station - Huron                  30
Lake Erie near Crane Reef                          25
Lake Erie off Lakeside (Marblehead)                24
Lake Erie @ City of Huron WTP Intake               22
Lake Erie Off Detroit Near Canadian Border         22
Lake Erie @ City of Vermilion WTP Intake           22
Lake Erie @ City of Sandusky WTP Intake            21
Lake Erie Ambient Site - Off Sandusky Bay          17
Lake Erie Ambient Station - Lorain West            13
Lake Erie Ambient Station, Off Cedar Point         13
Lake Erie Ambient Station - 

In [25]:
coords = oh_df.groupby(['Lat', 'Long']).agg({'Site':'first', 'Sample ID':'count'}).rename(columns={'Sample ID': 'Counts'}).reset_index()
coords

,Lat,Long,Site,Counts
0,41.405567,-82.557033,Lake Erie @ City of Huron WTP Intake,22
1,41.428333,-82.365033,Lake Erie @ City of Vermilion WTP Intake,22
2,41.444850,-82.454360,Lake Erie Ambient Station - Huron,30
3,41.459683,-82.641567,Lake Erie @ City of Sandusky WTP Intake,21
4,41.486570,-82.238770,Lake Erie Ambient Station - Lorain West,13
5,41.527740,-82.600620,"Lake Erie Ambient Station, Off Cedar Point",30
6,41.556749,-82.785072,Lake Erie off Lakeside (Marblehead),24
7,41.557810,-82.936350,Lake Erie North of Port Clinton,32
8,41.557817,-82.936350,Lake Erie North of Port Clinton,2
9,41.604040,-81.584450,Lake Erie Ambient Station - Wildwood,11


In [26]:
m = folium.Map(location=[41.76, -83.26])
for i in coords.index:
    lat = coords.loc[i, 'Lat']
    long = coords.loc[i, 'Long']
    site = coords.loc[i,'Site']
    counts = coords.loc[i, 'Counts']
    folium.Marker([lat, long], popup=(lat,long), tooltip=(site, counts)).add_to(m)
m

In [27]:
#restrict analysis to the western basin?
w_oh = oh_df[oh_df['Long'] < -82.7].copy()

In [28]:
w_coords = w_oh.groupby(['Lat', 'Long']).agg({'Site':'first', 'Sample ID':'count'}).rename(columns={'Sample ID': 'Counts'}).reset_index()

In [29]:
m = folium.Map(location=[41.76, -83.26])
for i in w_coords.index:
    lat = w_coords.loc[i, 'Lat']
    long = w_coords.loc[i, 'Long']
    site = w_coords.loc[i,'Site']
    counts = w_coords.loc[i, 'Counts']
    folium.Marker([lat, long], popup=(lat,long), tooltip=(site, counts)).add_to(m)
m

In [22]:
oh_df.columns

Index(['Site', 'Site ID', 'Site Type', 'Collect Date', 'Collection Time',
       'Sample ID', 'Parameter', 'Result', 'Units', 'Qual.*', 'Program', 'Lab',
       'Lat', 'Long', 'Saturation'],
      dtype='object')

In [34]:
glenda_df = pd.read_csv('../data/interim/glenda.csv', index_col=0, parse_dates=['SAMPLING_DATE'])
glenda_df

,Row,YEAR,MONTH,SEASON,LAKE,CRUISE_ID,VISIT_ID,STATION_ID,STN_DEPTH_M,LATITUDE,...,Extracted Chlorophyll a (µg/L),CTD Specific Conductivity (µS/cm),Total Nitrogen (mg/L),Nitrate + Nitrite (mg N/L),CTD Dissolved Oxygen (mg/L),Total Phosphorus (µg P/L),Secchi Depth (m),Total Suspended Solids (mg/L),CTD Temperature (°C),Turbidity (NTU)
0,1.0,2010.0,April,Spring,Erie,ER1011,E091M10,ER91M,10.2,41.838350,...,NaN,NaN,NaN,0.477,NaN,10.2165,NaN,NaN,7.1,NaN
1,2.0,2010.0,April,Spring,Erie,ER1011,E060M10,ER60,8.3,41.891750,...,NaN,232.9,NaN,0.401,NaN,7.2490,NaN,NaN,8.7,1.61
2,3.0,2010.0,April,Spring,Erie,ER1011,E061M10,ER61,8.9,41.946767,...,NaN,NaN,NaN,0.904,NaN,11.0665,NaN,NaN,8.7,NaN
3,4.0,2010.0,April,Spring,Erie,ER1011,E060M10,ER60,8.3,41.891750,...,NaN,NaN,NaN,0.401,NaN,7.4660,NaN,NaN,8.6,NaN
4,5.0,2010.0,April,Spring,Erie,ER1011,E059M10,ER59,8.0,41.727350,...,NaN,NaN,NaN,0.632,NaN,8.2365,NaN,NaN,7.7,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2811,2812.0,2018.0,April,Spring,Erie,ER1811,E91MM18,ER91M,9.5,41.840517,...,NaN,NaN,NaN,NaN,NaN,NaN,0.5,NaN,NaN,NaN
2812,2813.0,2019.0,April,Spring,Erie,ER1911,E958nA19,ER958n,10.3,41.525133,...,NaN,NaN,NaN,NaN,NaN,NaN,0.4,NaN,NaN,NaN
2813,2814.0,2019.0,April,Spring,Erie,ER1911,E959nA19,ER959n,10.5,42.186600,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
2814,2815.0,2019.0,April,Spring,Erie,ER1911,EER95nA19,ER95B,15.8,42.000052,...,NaN,NaN,NaN,NaN,NaN,NaN,3.8,NaN,NaN,NaN


In [35]:
glenda_coords = glenda_df.groupby(['LATITUDE', 'LONGITUDE']).agg({'STATION_ID': 'first', 'SAMPLING_DATE': 'count'}).rename(columns={'SAMPLING_DATE':'Counts'}).reset_index()
glenda_coords

,LATITUDE,LONGITUDE,STATION_ID,Counts
0,41.116667,-81.249667,ER78M,9
1,41.483653,-82.181587,ER964n,1
2,41.483803,-82.182537,ER964n,1
3,41.525133,-81.708467,ER958n,2
4,41.525350,-82.707887,ER958n,1
...,...,...,...,...
412,42.698050,-80.206003,ER937n,1
413,42.715067,-80.728405,ER937n,1
414,42.791000,-79.209000,ER932n,1
415,42.989783,-82.458203,ERG09n,1


In [42]:
m = folium.Map(location=[41.76, -83.26])
for i in w_coords.index:
    lat = w_coords.loc[i, 'Lat']
    long = w_coords.loc[i, 'Long']
    site = w_coords.loc[i,'Site']
    counts = w_coords.loc[i, 'Counts']
    folium.Marker([lat, long], popup=(lat,long), tooltip=(site, counts)).add_to(m)
for i in glenda_coords.index:
    lat = glenda_coords.loc[i, 'LATITUDE']
    long = glenda_coords.loc[i, 'LONGITUDE']
    site = glenda_coords.loc[i,'STATION_ID']
    counts = glenda_coords.loc[i, 'Counts']
    folium.Marker([lat, long], popup=(lat,long), tooltip=(site, counts), icon=folium.Icon(prefix='fa', icon='ship', color='red')).add_to(m)
m

In [43]:
import geopy.distance

In [47]:
coords_1 = (41.557817, -82.93635)
coords_2 = (41.5828, -82.91708299999998)
geopy.distance.distance(coords_1, coords_2).miles

1.9924168735389916

In [58]:
coords_dict = dict()
for i in coords.index:
    site = coords.loc[i, 'Site']
    coords_1 = (coords.loc[i, 'Lat'], coords.loc[i, 'Long'])
    for j in glenda_coords.index:
        coords_2 = (glenda_coords.loc[j, 'LATITUDE'], glenda_coords.loc[j, 'LONGITUDE'])
        if geopy.distance.distance(coords_1, coords_2).miles < 2.0:
            if site in coords_dict.keys():
                coords_dict[site].append(coords_2)
            else:
                coords_dict[site] = list()
                coords_dict[site].append(coords_2)
coords_dict

{'Lake Erie North of Port Clinton': [(41.5828, -82.91708299999998),
  (41.58325, -82.9183),
  (41.5828, -82.91708299999998),
  (41.58325, -82.9183)],
 'Lake Erie Ambient Site - Port Clinton': [(41.678533, -82.93181700000002),
  (41.683917, -82.9349),
  (41.684, -82.932),
  (41.6844, -82.93306700000002),
  (41.684683, -82.93295),
  (41.684767, -82.933667),
  (41.68485, -82.9337),
  (41.68495, -82.933483),
  (41.68495, -82.933317),
  (41.685, -82.934),
  (41.685, -82.93299999999998),
  (41.68515, -82.934483),
  (41.685233, -82.9341),
  (41.68535, -82.93395),
  (41.6854, -82.93423299999998),
  (41.68615, -82.93215),
  (41.6865, -82.947),
  (41.68655, -82.932417)],
 'Lake Erie Ambient Station - West Sister Island': [(41.718, -83.15),
  (41.725318, -83.15605),
  (41.7257, -83.152067),
  (41.725783, -83.150183),
  (41.726135, -83.149187),
  (41.72658300000001, -83.14935),
  (41.7268, -83.149967),
  (41.726933, -83.151933),
  (41.72695, -83.15015),
  (41.727, -83.15),
  (41.727, -83.149),
  (